<H2> Yash Pragnesh Gandhi <br><br> Github id: yash4gandhi <br><br> USC ID: 1150578261

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC,LinearSVC

In [97]:
from sklearn.metrics import hamming_loss, accuracy_score, silhouette_score, calinski_harabasz_score
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans


<h3>1. Multi-class and Multi-Label Classification Using Support Vector Machines
(a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.
uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data
randomly as the training set

In [27]:
df = pd.read_csv('../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')

In [28]:
df.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [29]:
l = list(df.columns)
l.remove('Family')
l.remove('Genus')
l.remove('Species')
l.remove('RecordID')

In [30]:
train,test = train_test_split(df,train_size=0.7, random_state = 15)

<h3>b) Each instance has three labels: Families, Genus, and Species. Each of the labels
has multiple classes. We wish to solve a multi-class and multi-label problem.
One of the most important approaches to multi-label classification is to train a
classifier for each label (binary relevance). We first try this approach:<br>
i. Research exact match and hamming score/ loss methods for evaluating multi-
label classification and use them in evaluating the classifiers in this problem.

Exact Match Loss refers to the ratio of number of incorrectly predicted classes to total number of points in the dataset.<br>
Hamming Loss refers to the average hamming distance. Hamming distance is the number of incorrect labels between two samples

<h3>ii. Train a SVM for each of the labels, using Gaussian kernels and one versus
all classifiers. Determine the weight of the SVM penalty and the width of
the Gaussian Kernel using 10 fold cross validation.1 You are welcome to try
to solve the problem with both standardized 2 and raw attributes and report
the results.

<h4> References:<br></h4>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html<br>
https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html<br>
https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/<br>
https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html<br>

<h3>SVM Classification for Family label

In [31]:
train_X = train[l]
test_X = test[l]
train_y = train[['Family']]
test_y = test[['Family']]

In [39]:
param_set = {'C': np.linspace(1, 3, 10), 'gamma' : np.linspace(0.1, 2, 10)}
gs = GridSearchCV(SVC(kernel='rbf',decision_function_shape = 'ovr'), param_grid = param_set , cv = KFold(10))
gs.fit(train_X, train_y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=SVC(),
             param_grid={'C': array([1.        , 1.22222222, 1.44444444, 1.66666667, 1.88888889,
       2.11111111, 2.33333333, 2.55555556, 2.77777778, 3.        ]),
                         'gamma': array([0.1       , 0.31111111, 0.52222222, 0.73333333, 0.94444444,
       1.15555556, 1.36666667, 1.57777778, 1.78888889, 2.        ])})

In [40]:
print('Best C: ',gs.best_params_['C'])

Best C:  3.0


In [41]:
print('Best gamma: ',gs.best_params_['gamma'])

Best gamma:  2.0


In [42]:
pred_y = gs.predict(test_X)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9893469198703103
Exact match Loss: 0.010653080129689685
Hamming Loss: 0.010653080129689671


<h3>SVM Classification for Genus label

In [43]:
train_y = train[['Genus']]
test_y = test[['Genus']]

In [44]:
param_set = {'C': np.linspace(1, 3, 10), 'gamma' : np.linspace(0.1, 2, 10)}
gs = GridSearchCV(SVC(kernel='rbf',decision_function_shape = 'ovr'), param_set , cv = KFold(10))
gs.fit(train_X, train_y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=SVC(),
             param_grid={'C': array([1.        , 1.22222222, 1.44444444, 1.66666667, 1.88888889,
       2.11111111, 2.33333333, 2.55555556, 2.77777778, 3.        ]),
                         'gamma': array([0.1       , 0.31111111, 0.52222222, 0.73333333, 0.94444444,
       1.15555556, 1.36666667, 1.57777778, 1.78888889, 2.        ])})

In [45]:
print('Best C: ',gs.best_params_['C'])

Best C:  3.0


In [46]:
print('Best gamma: ',gs.best_params_['gamma'])

Best gamma:  1.788888888888889


In [48]:
pred_y = gs.predict(test_X)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9884205650764243
Exact match Loss: 0.011579434923575715
Hamming Loss: 0.01157943492357573


<h3>SVM Classification for Species label

In [49]:
train_y = train[['Species']]
test_y = test[['Species']]

In [50]:
param_set = {'C': np.linspace(1, 3, 10), 'gamma' : np.linspace(0.1, 2, 10)}
gs = GridSearchCV(SVC(kernel='rbf',decision_function_shape = 'ovr'), param_set , cv = KFold(10))
gs.fit(train_X, train_y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=SVC(),
             param_grid={'C': array([1.        , 1.22222222, 1.44444444, 1.66666667, 1.88888889,
       2.11111111, 2.33333333, 2.55555556, 2.77777778, 3.        ]),
                         'gamma': array([0.1       , 0.31111111, 0.52222222, 0.73333333, 0.94444444,
       1.15555556, 1.36666667, 1.57777778, 1.78888889, 2.        ])})

In [51]:
print('Best C: ',gs.best_params_['C'])

Best C:  2.333333333333333


In [52]:
print('Best gamma: ',gs.best_params_['gamma'])

Best gamma:  1.3666666666666667


In [53]:
pred_y = gs.predict(test_X)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9888837424733673
Exact match Loss: 0.0111162575266327
Hamming Loss: 0.0111162575266327


<h3>iii. Repeat 1(b)ii with L1-penalized SVMs.3 Remember to standardize4 the at
tributes. Determine the weight of the SVM penalty using 10 fold cross vali-
dation.

In [54]:
s = preprocessing.StandardScaler()
train_scaled = s.fit_transform(train_X)
test_scaled = s.fit_transform(test_X)

<h3>SVM L1-penalized Classification for Family label

In [55]:
train_y = train[['Family']]
test_y = test[['Family']]

In [56]:
param_set = {'C': np.linspace(1, 3, 10)}

In [57]:
linearl1svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
gs = GridSearchCV(linearl1svm, param_set , cv= KFold(10)) 
gs.fit(train_scaled, train_y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=LinearSVC(dual=False, max_iter=5000, penalty='l1'),
             param_grid={'C': array([1.        , 1.22222222, 1.44444444, 1.66666667, 1.88888889,
       2.11111111, 2.33333333, 2.55555556, 2.77777778, 3.        ])})

In [58]:
print('Best C: ',gs.best_params_['C'])

Best C:  1.0


In [59]:
pred_y = gs.predict(test_scaled)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9258916164891153
Exact match Loss: 0.07410838351088467
Hamming Loss: 0.07410838351088467


<h3>SVM L1-penalized Classification for Genus label

In [60]:
train_y = train[['Genus']]
test_y = test[['Genus']]

In [61]:
linearl1svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
gs = GridSearchCV(linearl1svm, param_set , cv= KFold(10)) 
gs.fit(train_scaled, train_y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=LinearSVC(dual=False, max_iter=5000, penalty='l1'),
             param_grid={'C': array([1.        , 1.22222222, 1.44444444, 1.66666667, 1.88888889,
       2.11111111, 2.33333333, 2.55555556, 2.77777778, 3.        ])})

In [62]:
print('Best C: ',gs.best_params_['C'])

Best C:  1.6666666666666665


In [63]:
pred_y = gs.predict(test_scaled)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9407132931912923
Exact match Loss: 0.059286706808707734
Hamming Loss: 0.059286706808707734


<h3>SVM L1-penalized Classification for Species label

In [64]:
train_y = train[['Species']]
test_y = test[['Species']]

In [65]:
param_set = {'C': np.logspace(-1, 4, 20)}
linearl1svm = LinearSVC(penalty= 'l1', dual=False, max_iter = 5000)
gs = GridSearchCV(linearl1svm, param_set , cv= KFold(10)) 
gs.fit(train_scaled, train_y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=LinearSVC(dual=False, max_iter=5000, penalty='l1'),
             param_grid={'C': array([1.00000000e-01, 1.83298071e-01, 3.35981829e-01, 6.15848211e-01,
       1.12883789e+00, 2.06913808e+00, 3.79269019e+00, 6.95192796e+00,
       1.27427499e+01, 2.33572147e+01, 4.28133240e+01, 7.84759970e+01,
       1.43844989e+02, 2.63665090e+02, 4.83293024e+02, 8.85866790e+02,
       1.62377674e+03, 2.97635144e+03, 5.45559478e+03, 1.00000000e+04])})

In [66]:
print('Best C: ',gs.best_params_['C'])

Best C:  0.6158482110660264


In [67]:
pred_y = gs.predict(test_scaled)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9467345993515517
Exact match Loss: 0.05326540064844831
Hamming Loss: 0.053265400648448355


<h3>iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy
class imbalance. Report your conclusions about the classifiers you trained

In [68]:
sm = SMOTE()

In [72]:
sm_X_family, sm_y_family = sm.fit_resample(train_scaled, train['Family'])
sm_X_genus, sm_y_genus = sm.fit_resample(train_scaled, train['Genus'])
sm_X_species, sm_y_species = sm.fit_resample(train_scaled, train['Species'])

<h3>SMOTE SVM L1-penalized Classification for Family label

In [73]:
test_y = test[['Family']]

In [83]:
linearl1svm = LinearSVC(penalty= 'l1', multi_class = 'crammer_singer')
gs = GridSearchCV(linearl1svm, param_grid = {'C': np.linspace(1, 3, 5)} , cv= KFold(10)) 
gs.fit(sm_X_family, sm_y_family)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=LinearSVC(multi_class='crammer_singer', penalty='l1'),
             param_grid={'C': array([1. , 1.5, 2. , 2.5, 3. ])})

In [84]:
print('Best C: ',gs.best_params_['C'])

Best C:  1.0


In [85]:
pred_y = gs.predict(test_scaled)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9235757295044001
Exact match Loss: 0.07642427049559986
Hamming Loss: 0.07642427049559981


<h3>SMOTE SVM L1-penalized Classification for Genus label

In [86]:
test_y = test[['Genus']]

In [87]:
linearl1svm = LinearSVC(penalty= 'l1', multi_class = 'crammer_singer')
gs = GridSearchCV(linearl1svm, param_grid = {'C': np.linspace(1, 3, 5)} , cv= KFold(10)) 
gs.fit(sm_X_genus, sm_y_genus)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=LinearSVC(multi_class='crammer_singer', penalty='l1'),
             param_grid={'C': array([1. , 1.5, 2. , 2.5, 3. ])})

In [88]:
print('Best C: ',gs.best_params_['C'])

Best C:  3.0


In [89]:
pred_y = gs.predict(test_scaled)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9416396479851783
Exact match Loss: 0.058360352014821704
Hamming Loss: 0.058360352014821676


<h3>SMOTE SVM L1-penalized Classification for Species label

In [90]:
test_y = test[['Species']]

In [91]:
linearl1svm = LinearSVC(penalty= 'l1', multi_class = 'crammer_singer')
gs = GridSearchCV(linearl1svm,  param_grid = {'C': np.linspace(1, 3, 5)}  , cv= KFold(10)) 
gs.fit(sm_X_species, sm_y_species)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
             estimator=LinearSVC(multi_class='crammer_singer', penalty='l1'),
             param_grid={'C': array([1. , 1.5, 2. , 2.5, 3. ])})

In [92]:
print('Best C: ',gs.best_params_['C'])

Best C:  3.0


In [93]:
pred_y = gs.predict(test_scaled)
accuracy = accuracy_score(test_y, pred_y)
hammingloss = hamming_loss(test_y, pred_y)
print('Accuracy :',accuracy)
print('Exact match Loss:', 1-accuracy)
print('Hamming Loss:', hammingloss)

Accuracy : 0.9615562760537286
Exact match Loss: 0.03844372394627138
Hamming Loss: 0.03844372394627142


<h3> Based on the above values we can see that normal SVM classification i.e 1b: ii) gives maximum accuracy of arrounf 98% for all labels.

<h3>2. K-Means Clustering on a Multi-Class and Multi-Label Data Set
Monte-Carlo Simulation: Perform the following procedures 50 times, and report
the average and standard deviation of the 50 Hamming Distances that you calculate<br>
(a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split
the data into train and test, as we are not performing supervised learning in this
exercise). Choose k ∈ {1, 2, . . . , 50} automatically based on one of the methods
provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any
other method you know.

In [94]:
df_X = df[l]
df_y = df[['Family', 'Genus', 'Species']]

result = [['Iteration','Best K value']]

In [102]:
import operator

In [117]:
i = 0
while(i<50):
    score = {}
    k = 2
    while(k<=50):
        km = KMeans(n_clusters=k)
        km = km.fit(df_X)
        score[k] = silhouette_score(df_X, km.labels_)
        k+=1
    print('Iteration: ',i)    
    bestscore = sorted(score.items(), key=operator.itemgetter(1), reverse=True)
    #print(bestscore)
    bestk = bestscore[0][0]
    result.append([i+1,bestk])
    i+=1

Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49


In [127]:
resultdf = pd.DataFrame(result[1:],columns=result[0])

In [128]:
print(resultdf)

    Iteration  Best K value
0           1             4
1           2             4
2           3             4
3           4             4
4           5             4
5           6             4
6           7             4
7           8             4
8           9             4
9          10             4
10         11             4
11         12             4
12         13             4
13         14             4
14         15             4
15         16             4
16         17             4
17         18             4
18         19             4
19         20             4
20         21             4
21         22             4
22         23             4
23         24             4
24         25             4
25         26             4
26         27             4
27         28             4
28         29             4
29         30             4
30         31             4
31         32             4
32         33             4
33         34             4
34         35       

<h3>(b) In each cluster, determine which family is the majority by reading the true labels.
Repeat for genus and species.

<h3>(c) Now for each cluster you have a majority label triplet (family, genus, species).
Calculate the average Hamming distance, Hamming score, and Hamming loss5
between the true labels and the labels assigned by clusters.

References:
https://www.statology.org/hamming-distance-python/
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
https://wiki.hasty.ai/metrics/hamming-score#:~:text=Hamming%20Score%20is%20the%20fraction,in%20fact%20they%20are%20interchangeable.

In [129]:
from scipy.spatial.distance import hamming

In [ ]:
lis = [[1,4]]

In [144]:
classMajority = [['Iteration','Cluster Number','Family','Genus','Species']]
classLoss = [['Iteration','Average_Triplet_Hamming_Loss','Average_Triplet_Hamming_Distance','Average_Triplet_Hamming_Score']]
print('Iteration:')
for el in result:   
    if el[1] != 'Best K value':
        print(el[0])
        km = KMeans(n_clusters=el[1])
        km = km.fit(df_X)
        df['k_value'] = km.labels_
        k=0
        while k < el[1]:
            family = df[df.k_value == k]['Family'].value_counts().index[0]  
            genus = df[df.k_value == k]['Genus'].value_counts().index[0]
            species = df[df.k_value == k]['Species'].value_counts().index[0]
            
            classMajority.append([el[0],k+1,family,genus,species])
            
            df.loc[df.k_value == k, 'Cluster_Assigned_Family'] = family
            df.loc[df.k_value == k, 'Cluster_Assigned_Genus'] = genus
            df.loc[df.k_value == k, 'Cluster_Assigned_Species'] = species
            
            k+=1
            
        
        lossF = hamming_loss(df['Family'], df['Cluster_Assigned_Family'])
        lossG = hamming_loss(df['Genus'], df['Cluster_Assigned_Genus'])
        lossS = hamming_loss(df['Species'], df['Cluster_Assigned_Species'])
        avg_loss = (lossF + lossG + lossS)/3
        
        accF = accuracy_score(df['Family'], df['Cluster_Assigned_Family'])
        accG = accuracy_score(df['Genus'], df['Cluster_Assigned_Genus'])
        accS = accuracy_score(df['Species'], df['Cluster_Assigned_Species'])
        avg_acc = (accF + accG + accS)/3
        
        hamming_dist = 0
        for row in range(df.shape[0]):
            x = list(df[['Family','Genus','Species']].iloc[row])
            y = list(df[['Cluster_Assigned_Family','Cluster_Assigned_Genus','Cluster_Assigned_Species']].iloc[row])
            hamming_dist = hamming_dist + hamming(x, y) * len(x)
        
        avg_hamming_dist = hamming_dist/df.shape[0]
        classLoss.append([el[0],avg_loss,avg_hamming_dist,avg_acc])
            

Iteration:
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [145]:
classMajoritydf = pd.DataFrame(classMajority[1:],columns=classMajority[0])
classMajoritydf

,Iteration,Cluster Number,Family,Genus,Species
0,1,1,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,1,2,Hylidae,Hypsiboas,HypsiboasCinerascens
2,1,3,Hylidae,Hypsiboas,HypsiboasCordobae
3,1,4,Dendrobatidae,Ameerega,Ameeregatrivittata
4,2,1,Hylidae,Hypsiboas,HypsiboasCordobae
...,...,...,...,...,...
195,49,4,Hylidae,Hypsiboas,HypsiboasCinerascens
196,50,1,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
197,50,2,Hylidae,Hypsiboas,HypsiboasCordobae
198,50,3,Hylidae,Hypsiboas,HypsiboasCinerascens


In [146]:
classLossdf = pd.DataFrame(classLoss[1:],columns=classLoss[0])
classLossdf

,Iteration,Average_Triplet_Hamming_Loss,Average_Triplet_Hamming_Distance,Average_Triplet_Hamming_Score
0,1,0.222423,0.667269,0.777577
1,2,0.222469,0.667408,0.777531
2,3,0.222423,0.667269,0.777577
3,4,0.221913,0.665740,0.778087
4,5,0.222145,0.666435,0.777855
5,6,0.222423,0.667269,0.777577
6,7,0.222423,0.667269,0.777577
7,8,0.222423,0.667269,0.777577
8,9,0.221774,0.665323,0.778226
9,10,0.222423,0.667269,0.777577


In [147]:
hammingdistance = classLossdf['Average_Triplet_Hamming_Distance'].to_numpy()
print('Final Average Hamming Distance',np.mean(hammingdistance))
print('Final Standard Deviation of Hamming Distance',np.std(hammingdistance))

Final Average Hamming Distance 0.6717804030576788
Final Standard Deviation of Hamming Distance 0.015391608384660126


<h3>3. ISLR 12.6.2

<h4> Answers are attached in a pdf and uploaded under notebook directory